## Image Segmentation with SLIC and clustering with k-means
### Abstract
In this project we implement k-means method for clustering a given set of points in two
different coordinates : Cartesian and polar coordinates. Functions for this are in
`k_means.py`. Also, we will implement SLIC method for image segmentation. Functions
for SLIC are in `slic.py`. `main.py` contains code for testing these functions.


### k-means
We will implement k-means method for clustering
a given set of points in plane. for all results, we set
k=2, but code can be used for clustering points to any
number of clusters.
<div>
<ol>
<li>
First two results are clustering of points based on their
coordinates in Euclidean system. Clustering points in
this way divides them (almost) equally into two sets, by
a line passing throw the center of the coordinates. This would
be a random line, as the initial cluster centers are chosen
randomly.
<li>
Last result is a clustering
of points based on their magnitude in polar coordinates,
which is a better clustering. This method will divide the points
into two sets, one of them being the smaller circle arround
the center, and the second being the ring of point arround the center
which is farther from central points. This result seems to be a better results,
since the points in different clusters are far from each other and indeed seem to be
in different components.
</ol>
</div>

The function `readPoints` is used to read the coordinates
of points written in a given file,and returns a numpy array containing them:

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import math


def readPoints(file):
    f = open(file)
    n = int(f.readline())
    points = np.zeros((2, 0))
    for line in f.readlines():
        point = line.split(" ")
        [x, y] = float(point[0]), float(point[1])
        points = np.c_[points, [x, y]]
    return n, points



next function initializes k random centers from a
given set of points :

In [ ]:
def get_random_centers(points, k):
    mask = np.random.choice(np.arange(points.shape[1]), k, replace=False)
    return points[:, mask]

The `cluster_points` function takes points, cluster centers and integer i
as parameters, and returns the set of points which fall into the i'th cluster:

In [ ]:
def cluster_points(points, selected_points, i):  # returns an array containing the i'th cluster
    result = np.zeros((points.shape[0], 0))
    for j in range(points.shape[1]):
        m = np.sum(np.square(points[:, j] - selected_points[:, i]))
        is_in_cluster = True
        for x in range(selected_points.shape[1]):
            n = np.sum(np.square(points[:, j] - selected_points[:, x]))
            if n < m:
                is_in_cluster = False
                break
        if is_in_cluster:
            result = np.c_[result, points[:, j]]

    return result

Function `get_mean_points` takes the points of a cluster as
input and outputs the new center (mean point) of the cluster:

In [ ]:
def get_mean_point(cluster):
    mean = np.average(cluster, axis=1)
    return mean


Main function for the first two results of the code is
function `k-mean`. It takes points and k ( and a threshold, which is set to be zero )
as input and using the previous functions outputs the final cluster centers of the points ( which is an array of
k points )

In [ ]:
def k_means(points, k, threshold=0):
    selected_points = get_random_centers(points, k)
    diff = float('inf')
    while diff > threshold:
        mean_points = np.zeros((points.shape[0], k))
        for i in range(k):
            cluster_i = cluster_points(points, selected_points, i)
            mean_i = get_mean_point(cluster_i)
            mean_points[:, i] = mean_i
        diff = np.sum(np.square(selected_points - mean_points))
        selected_points = mean_points.copy()
    return mean_points


Finally, function `draw_fig`, takes points, cluster centers,k, a list of colors
( for coloring points in each cluster) and an output path, and saves the image of clustered
points in the given output path:

In [ ]:
def draw_fig(points, cluster_centers, k, colors, output_path):
    for i in range(k):
        cluster_i = cluster_points(points, cluster_centers, i)
        for j in range(cluster_i.shape[1]):
            plt.plot(cluster_i[0, j], cluster_i[1, j], marker='.', color=colors[i])

    plt.savefig(output_path)
    plt.show()

### Clustering in Polar Coordinates
Following functions are used to cluster points based on their
magnitude in polar coordinates. Main implementation of them
is similar to the previous functions: we first take the given points
to polar space and apply k-mean on their magnitude, just in the same way we applied it
for Euclidean coordinates.

In [ ]:
def get_polar_coordinates(points):
    polar_points = np.zeros(points.shape)
    for i in range(points.shape[1]):
        x, y = points[0, i], points[1, i]
        r = np.sqrt(x ** 2 + y ** 2)
        theta = math.atan2(x, y)
        polar_points[:, i] = (r, theta)
    return polar_points


def cluster_points_polar(points, selected_points, i):
    result = np.zeros((points.shape[0], 0))
    for j in range(points.shape[1]):
        m = np.sum(np.square(points[0, j] - selected_points[0, i]))
        is_in_cluster = True
        for x in range(selected_points.shape[1]):
            n = np.sum(np.square(points[0, j] - selected_points[0, x]))
            if n < m:
                is_in_cluster = False
                break
        if is_in_cluster:
            result = np.c_[result, points[:, j]]

    return result


def k_means_polar(points, k, threshold=0):
    selected_points = get_random_centers(points, k)
    diff = float('inf')
    while diff > threshold:
        mean_points = np.zeros((points.shape[0], k))
        for i in range(k):
            cluster_i = cluster_points_polar(points, selected_points, i)
            mean_i = get_mean_point(cluster_i)
            mean_points[:, i] = mean_i
        diff = np.sum(np.square(selected_points[0:] - mean_points[0:]))
        selected_points = mean_points.copy()
    return mean_points


Final function is used to draw points in polar coordinates in plane, and save the resulting image
in a given directory:

In [ ]:
def draw_polar_fig(polar_points, polar_centers, k, colors, output_path):
    for i in range(k):
        cluster_i = cluster_points_polar(polar_points, polar_centers, i)
        for j in range(cluster_i.shape[1]):
            r, theta = cluster_i[0, j], cluster_i[1, j]
            x, y = r * np.sin(theta), r * np.cos(theta)
            plt.plot(x, y, marker='.', color=colors[i])

    plt.savefig(output_path)
    plt.show()

### The Main File
Now in `main.py`, we will use the functions in `k_means.py` to apply k-mean.
We start by reading points from file and plotting them:

In [ ]:
import k_means
import matplotlib.pyplot as plt
import numpy as np
import cv2

input_path = "inputs/Points.txt"
output_paths = ["outputs/res01.jpg", "outputs/res02.jpg", "outputs/res03.jpg", "outputs/res04.jpg"]

''' showing initial points '''

n, points = k_means.readPoints("inputs/Points.txt")
for i in range(points[0].size):
    plt.plot(points[0, i], points[1, i], marker='.', color='blue')
plt.savefig(output_paths[0])
plt.show()

Then by setting k=2 and color of each cluster, we apply first k-mean method
(on Euclidean coordinates ) 2 times :

In [ ]:
k = 2
colors = ["red", "green"]
for i in range(1, 3):
    selected_points = k_means.get_random_centers(points, k)
    centers = k_means.k_means(points, k)
    k_means.draw_fig(points, centers, k, colors, output_paths[i])

Finally, we cluster points based on their polar coordinates, by first taking them
to polar space, and then applying k-mean on the magnitude of the points:

In [ ]:
''' clustering in polar coordinates '''
polar_points = k_means.get_polar_coordinates(points)
polar_centers = k_means.k_means_polar(polar_points, k)
k_means.draw_polar_fig(polar_points, polar_centers, k, colors, output_paths[3])


### SLIC method
